## Real Time Testing

In [1]:
! py -3.9 -m pip install cvzone==1.5.6
! py -3.9 -m pip install opencv-python==4.7.0.72
! py -3.9 -m pip install numpy==1.26.2
! py -3.9 -m pip install mediapipe==0.9.1.0
! py -3.9 -m pip install tensorflow==2.11.0
! py -3.9 -m pip install wordninja==2.0.0
! py -3.9 -m pip install transformers==4.35.2
! py -3.9 -m pip install kivy==2.1.0
! py -3.9 -m pip install kivymd==1.1.1
! py -3.9 -m pip install gtts
! py -3.9 -m pip install deep-translator
! py -3.9 -m pip install pyttsx3
! py -3.9 -m pip install playsound
! py -3.9 -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
! py -3.9 -m pip list

Using legacy 'setup.py install' for cvzone, since package 'wheel' is not installed.
    Running setup.py install for cvzone: started
    Running setup.py install for cvzone: finished with status 'done'


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.1.78
    Uninstalling opencv-python-4.8.1.78:
      Successfully uninstalled opencv-python-4.8.1.78


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


...

You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://download.pytorch.org/whl/cu121


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Package                      Version
---------------------------- ------------
absl-py                      2.0.0
asttokens                    2.4.1
astunparse                   1.6.3
attrs                        23.1.0
beautifulsoup4               4.12.2
cachetools                   5.3.2
certifi                      2023.11.17
charset-normalizer           3.3.2
click                        8.1.7
colorama                     0.4.6
comm                         0.2.0
comtypes                     1.2.0
contourpy                    1.2.0
cvzone                       1.5.6
cycler                       0.12.1
debugpy                      1.8.0
decorator                    5.1.1
deep-translator              1.11.4
docutils                     0.20.1
exceptiongroup               1.2.0
executing                    2.0.1
filelock                     3.13.1
flatbuffers                  23.5.26
fonttools                    4.46.0
fsspec                       2023.12.2
gast                        

You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
from time import time

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [3]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [4]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [5]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        # t2 = executor.submit(detectPose, poseDetector, frame, draw)
        # t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            # t2.result(), 
            # t3.result()['bbox'],
            # t3.result()['center'],
            # t3.result()['center'] - t1.result()[0]['center'],
            # t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [6]:
import keras
from files_io import readActionLabels

# Load the model from the H5 file
# model = keras.models.load_model('../static_recognition/models/static_model_ANN.h5')
model = keras.models.load_model('../static_recognition/models/static_model_V2.h5')
# model = keras.models.load_model('../static_recognition/models/static_model.keras')

# Print the summary before loading
model.summary()

static_labels = readActionLabels()
static_labels

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 136, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 68, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 31, 128)           24704     
                                                                 
 flatten_1 (Flatten)         (None, 3968)             

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10']

In [11]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)
fpsReader = FPS()

timeStats = []

try:
    predictionHistory = deque()
    detectionThreshold = 0.99999    
    predictionCooldown = 0.5
    appendCooldown = 1.0
    # the first time append should eliminate predictionCooldown
    lastAppendTime = time() + appendCooldown
    lastDetectTime = time() + predictionCooldown

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Hand Detection
        inputValue, frame = staticFeatureExtraction(
            handDetector, frame)
        
        # detectionResults, frame = featureExtractionV3(
        #     handDetector, faceDetector, poseDetector, frame)
        
        detectionResults = np.expand_dims(
            inputValue, axis=0) # reshape detection result
        
        if time() <= lastDetectTime + predictionCooldown:
            pass
        else:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]
            # lastDetectTime = time()
            
            print(predictionResults)
            
            # Get predicted character and accuracy
            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
        
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)          
            
            if predAccuracy >= detectionThreshold:            
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                # Check if 0.5 seconds have passed since the last append
                if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + appendCooldown:
                    # Do nothing, don't append
                    pass
                else:
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()

                if len(predictionHistory) > 5:
                        predictionHistory.popleft()  
                        
        # cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0),scale=5, thickness=5)

        # fps, frame = fpsReader.update(frame, pos=(
        #     950, 80), color=(0, 255, 0), scale=5, thickness=5)

        
        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()
            lastAppendTime = time() + predictionCooldown


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

[1.0500661e-06 2.6376615e-04 8.1490659e-10 7.8708117e-06 5.9649057e-07
 5.2042934e-04 2.0367722e-07 9.9588710e-01 1.3738385e-03 4.4132915e-07
 1.2279704e-04 1.7146708e-04 5.2469596e-07 4.4481876e-06 1.1703173e-07
 9.6846568e-05 1.6985978e-05 1.8801932e-06 1.5781659e-05 5.4274478e-05
 7.2673902e-05 1.1449739e-08 3.5130924e-08 5.9772083e-06 4.2220014e-05
 9.1425830e-04 2.9845498e-04 4.3565038e-07 3.3139251e-05 1.0776123e-05
 5.6186350e-06 2.0866366e-05 2.3681216e-06 1.3001511e-05 1.3187357e-06
 3.8504502e-05]
[1.0500661e-06 2.6376615e-04 8.1490659e-10 7.8708117e-06 5.9649057e-07
 5.2042934e-04 2.0367722e-07 9.9588710e-01 1.3738385e-03 4.4132915e-07
 1.2279704e-04 1.7146708e-04 5.2469596e-07 4.4481876e-06 1.1703173e-07
 9.6846568e-05 1.6985978e-05 1.8801932e-06 1.5781659e-05 5.4274478e-05
 7.2673902e-05 1.1449739e-08 3.5130924e-08 5.9772083e-06 4.2220014e-05
 9.1425830e-04 2.9845498e-04 4.3565038e-07 3.3139251e-05 1.0776123e-05
 5.6186350e-06 2.0866366e-05 2.3681216e-06 1.3001511e-05 1.31

In [8]:
timeStats[10:]

[0.08759212493896484,
 0.08055710792541504,
 0.08977508544921875,
 0.07969331741333008,
 0.10363626480102539,
 0.1076974868774414,
 0.09026694297790527,
 0.07944512367248535,
 0.11448287963867188,
 0.08859372138977051,
 0.10600042343139648,
 0.12326645851135254,
 0.11517620086669922,
 0.0946652889251709,
 0.09112691879272461,
 0.1170506477355957,
 0.09410905838012695,
 0.08658790588378906,
 0.10114455223083496,
 0.09341096878051758,
 0.09839892387390137,
 0.10508155822753906,
 0.08784675598144531,
 0.09278321266174316,
 0.10693788528442383,
 0.09531426429748535,
 0.1260986328125,
 0.1171731948852539,
 0.1251673698425293,
 0.12038397789001465,
 0.11404561996459961,
 0.11678814888000488,
 0.1272568702697754,
 0.11148285865783691,
 0.11443853378295898,
 0.14362335205078125,
 0.13240480422973633,
 0.12762713432312012,
 0.12087512016296387,
 0.1120905876159668,
 0.1786794662475586,
 0.12734246253967285,
 0.14897489547729492,
 0.1266937255859375,
 0.12304496765136719,
 0.1235036849975586,
 0